# Практическая работа
## Часть II. Генерация и анализ семантических данных

###  Основы семантических данных и графовых моделей знаний

В современных экономических исследованиях объемы данных стремительно растут, и традиционные табличные методы их хранения и анализа (как в Excel) не всегда позволяют выявлять сложные, неявные связи. На смену им приходят семантические технологии и графовые модели данных, которые представляют информацию в виде сети взаимосвязанных сущностей.

**Ключевые концепции:**

1.  **Семантические данные (Semantic Data):** это данные, структурированные таким образом, чтобы их смысл был понятен не только человеку, но и машине. Основная идея — описывать не только сами данные, но и отношения между ними. Вместо таблицы, где связь между столбцами определяется только их положением, семантический подход явным образом определяет каждую связь.

2.  **RDF (Resource Description Framework):** это стандарт W3C для представления данных в виде графа. Основной строительный блок RDF — это **триплет**, состоящий из трех компонентов:
    *   **Субъект (Subject):** ресурс, который мы описываем (например, "компания за 2017 год").
    *   **Предикат (Predicate):** свойство или тип отношения, которое связывает субъект и объект (например, "имеет доход").
    *   **Объект (Object):** значение свойства или другой ресурс (например, "350 млн. руб.").

    *Пример триплета:* `<Год_2017> <имеет_доход> "350"`.

3.  **Граф знаний (Knowledge Graph):** это сеть, состоящая из множества RDF-триплетов. Она позволяет моделировать сложные доменные области (например, экономику предприятия), где все сущности (годы, доходы, расходы, инвестиции) и их взаимосвязи представлены в виде узлов и ребер графа. Такой подход обеспечивает гибкость и масштабируемость анализа.

4.  **Онтологии (RDFS, OWL):** для придания данным единообразия и формализации используются онтологии — словари, которые определяют классы сущностей (например, `ФинансовыйПоказатель`, `ГодовойОтчет`) и типы отношений между ними (`имеетДоход`, `связанСИнвестициями`). Это позволяет стандартизировать описание данных и выполнять логический вывод новых знаний.

5.  **SPARQL (SPARQL Protocol and RDF Query Language):** это язык запросов, аналогичный SQL, но предназначенный для работы с RDF-графами. С помощью SPARQL можно извлекать данные, находить паттерны и выполнять сложные аналитические запросы к графу знаний, которые были бы затруднительны в реляционной модели.

В этой практической работе мы преобразуем табличные экономические данные в семантический RDF-граф и научимся извлекать из него знания с помощью запросов SPARQL.

## Шаг 1. Подготовка среды и данных

Сначала установим библиотеку `rdflib`, которая является основным инструментом в Python для работы с RDF-данными.

In [ ]:
!pip install rdflib

Теперь импортируем все необходимые модули и загрузим наши исходные данные в DataFrame библиотеки `pandas` для удобства.

In [ ]:
import pandas as pd
from rdflib import Graph, Literal, RDF, RDFS, Namespace, URIRef
from rdflib.namespace import XSD

# Исходные данные в виде словаря
data = {
    'Год': [2016, 2017, 2018, 2019, 2020, 2021, 2022],
    'Доходы, млн. руб.': [300, 350, 400, 420, 450, 480, 500],
    'Расходы, млн. руб.': [200, 210, 240, 250, 270, 290, 300],
    'Прибыль, млн. руб.': [100, 140, 160, 170, 180, 190, 200],
    'Инвестиции, млн. руб.': [50, 70, 90, 110, 120, 130, 140]
}
df = pd.DataFrame(data)

print("Исходные данные:")
print(df)

## Шаг 2. Определение онтологии (пространства имен)

Чтобы наши данные в графе были структурированными, мы определим пространства имен (namespaces). Это как префиксы для наших будущих URI.

*   `EX` (example) - для уникальных экземпляров наших данных (например, для отчета за конкретный год).
*   `SCHEMA` - для описания схемы данных: классов (типов сущностей) и предикатов (свойств).

In [3]:
# Пространство имен для наших данных
EX = Namespace("http://example.org/economicdata/")
# Пространство имен для нашей простой онтологии (свойств и классов)
SCHEMA = Namespace("http://example.org/schema#")

## Шаг 3. Создание и наполнение RDF-графа

На этом шаге мы:
1.  Создаем пустой граф `g`.
2.  Привязываем к нему наши префиксы для удобного вывода.
3.  Определяем базовую схему (онтологию): создаем класс `YearlyReport` (Годовой отчет) и свойства `income` (доход), `expenses` (расходы) и т.д.
4.  Проходим в цикле по каждой строке нашего DataFrame и преобразуем ее в набор RDF-триплетов, которые добавляем в граф.

In [ ]:
g = Graph()
g.bind("ex", EX)
g.bind("schema", SCHEMA)
g.bind("rdfs", RDFS)

# Определяем классы и свойства в нашей "схеме"
g.add((SCHEMA.YearlyReport, RDF.type, RDFS.Class))
g.add((SCHEMA.income, RDF.type, RDF.Property))
g.add((SCHEMA.expenses, RDF.type, RDF.Property))
g.add((SCHEMA.profit, RDF.type, RDF.Property))
g.add((SCHEMA.investments, RDF.type, RDF.Property))
g.add((SCHEMA.year, RDF.type, RDF.Property))

# Добавляем данные из DataFrame в граф
for index, row in df.iterrows():
    year = row['Год']
    # Создаем уникальный ресурс (URI) для каждого годового отчета
    report_uri = EX[f'report_{year}']

    # Добавляем триплеты в граф
    g.add((report_uri, RDF.type, SCHEMA.YearlyReport)) # Указываем, что это экземпляр класса YearlyReport
    g.add((report_uri, SCHEMA.year, Literal(year, datatype=XSD.gYear)))
    g.add((report_uri, SCHEMA.income, Literal(row['Доходы, млн. руб.'], datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.expenses, Literal(row['Расходы, млн. руб.'], datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.profit, Literal(row['Прибыль, млн. руб.'], datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.investments, Literal(row['Инвестиции, млн. руб.'], datatype=XSD.integer)))

print(f"Граф успешно создан. Количество триплетов: {len(g)}")

## Шаг 4. Сериализация и просмотр графа

"Сериализация" — это процесс преобразования графа из памяти в текстовый формат. Мы будем использовать формат `Turtle` (.ttl), так как он очень удобен для чтения человеком. Это позволит нам увидеть, как наши табличные данные теперь представлены в виде семантических связей.

In [ ]:
# Вывод графа в формате Turtle
print(g.serialize(format="turtle"))

## Шаг 5. Анализ данных с помощью SPARQL-запросов

SPARQL — это язык запросов к RDF-графам, аналог SQL для реляционных баз данных. С его помощью мы можем извлекать сложные срезы данных и находить неочевидные связи.

### Запрос 1: Найти все финансовые показатели за 2020 год.
Этот запрос находит узел, у которого свойство `schema:year` равно "2020", и извлекает все связанные с ним финансовые показатели.

In [ ]:
q1 = """
PREFIX schema: <http://example.org/schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?income ?expenses ?profit ?investments
WHERE {
    ?report schema:year "2020"^^xsd:gYear .
    ?report schema:income ?income .
    ?report schema:expenses ?expenses .
    ?report schema:profit ?profit .
    ?report schema:investments ?investments .
}
"""

print("Результаты запроса 1 (показатели за 2020 год):")
for row in g.query(q1):
    print(f"Доходы: {row.income}, Расходы: {row.expenses}, Прибыль: {row.profit}, Инвестиции: {row.investments}")

### Запрос 2: Найти годы, когда инвестиции превысили 100 млн. руб., и отсортировать их по убыванию прибыли.

Здесь мы используем `FILTER` для отбора данных по условию и `ORDER BY DESC()` для сортировки результатов.

In [ ]:
q2 = """
PREFIX schema: <http://example.org/schema#>

SELECT ?year ?profit ?investments
WHERE {
    ?report schema:year ?year .
    ?report schema:profit ?profit .
    ?report schema:investments ?investments .
    FILTER(?investments > 100)
}
ORDER BY DESC(?profit)
"""

print("\nРезультаты запроса 2 (годы с инвестициями > 100 млн. руб.):")
for row in g.query(q2):
    print(f"Год: {row.year}, Прибыль: {row.profit}, Инвестиции: {row.investments}")

### Задание для отчета: собственный аналитический запрос

Теперь ваша задача — составить собственный, более сложный запрос.

**Пример:** Рассчитать рентабельность (`Прибыль / Доходы`) прямо в запросе и вывести топ-3 самых рентабельных года. Для этого можно использовать вычисления внутри `SELECT` и оператор `LIMIT`.

In [ ]:
g = Graph()
g.bind("ex", EX)
g.bind("schema", SCHEMA)
g.bind("rdfs", RDFS)

# Определяем классы и свойства в нашей "схеме"
g.add((SCHEMA.YearlyReport, RDF.type, RDFS.Class))
g.add((SCHEMA.income, RDF.type, RDF.Property))
g.add((SCHEMA.expenses, RDF.type, RDF.Property))
g.add((SCHEMA.profit, RDF.type, RDF.Property))
g.add((SCHEMA.investments, RDF.type, RDF.Property))
g.add((SCHEMA.year, RDF.type, RDF.Property))

# Добавляем данные из DataFrame в граф
for index, row in df.iterrows():
    year = row['Год']
    report_uri = EX[f'report_{year}']

    # ИСПРАВЛЕНИЕ: Преобразуем numpy.int64 в стандартный int()
    g.add((report_uri, RDF.type, SCHEMA.YearlyReport))
    g.add((report_uri, SCHEMA.year, Literal(year, datatype=XSD.gYear)))
    g.add((report_uri, SCHEMA.income, Literal(int(row['Доходы, млн. руб.']), datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.expenses, Literal(int(row['Расходы, млн. руб.']), datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.profit, Literal(int(row['Прибыль, млн. руб.']), datatype=XSD.integer)))
    g.add((report_uri, SCHEMA.investments, Literal(int(row['Инвестиции, млн. руб.']), datatype=XSD.integer)))

print(f"Граф успешно создан. Количество триплетов: {len(g)}")

In [ ]:
q3 = """
PREFIX schema: <http://example.org/schema#>

SELECT ?year ?profit ?income ((?profit / ?income) * 100 AS ?profitability)
WHERE {
    ?report schema:year ?year .
    ?report schema:profit ?profit .
    ?report schema:income ?income .
}
ORDER BY DESC(?profitability)
LIMIT 3
"""

print("\nРезультаты запроса 3 (топ-3 рентабельных года):")

# Выполнение запроса и вывод результатов
for row in g.query(q3):
    # row.profitability - это rdflib.term.Literal, его нужно привести к float для форматирования
    print(f"Год: {row.year}, Рентабельность: {float(row.profitability):.2f}%")

In [ ]:
!pip install pyvis

In [ ]:
from pyvis.network import Network
import webbrowser
import os

# Создаем объект сети pyvis
net = Network(notebook=True, cdn_resources='in_line', height="600px", width="100%")

# Функция для сокращения длинных URI для наглядности
def short_name(uri):
    if isinstance(uri, URIRef):
        return uri.split('/')[-1].split('#')[-1]
    else:
        # Для Literal возвращаем само значение
        return str(uri)

# Добавляем узлы и ребра из нашего графа g
for s, p, o in g:
    subject_name = short_name(s)
    predicate_name = short_name(p)
    object_name = short_name(o)

    # Добавляем узлы
    net.add_node(subject_name, label=subject_name, title=str(s), color="#4E8CF5") # Субъекты - синие
    net.add_node(object_name, label=object_name, title=str(o), color="#F5A623")  # Объекты - оранжевые

    # Добавляем ребро
    net.add_edge(subject_name, object_name, label=predicate_name)

# Устанавливаем физические параметры для лучшего отображения
net.repulsion(node_distance=150, spring_length=200)

# Генерируем и отображаем HTML файл с графом
try:
    file_path = 'knowledge_graph.html'
    net.show(file_path)
    print(f"Интерактивный граф сохранен в файл: {file_path}")
    print("Откройте его, чтобы исследовать связи (в Google Colab файл появится в панели слева).")
except Exception as e:
    print(f"Ошибка при отображении графа: {e}")

## Шаг 7. Визуализация аналитических данных

Теперь визуализируем результаты нашего SPARQL-анализа. Построим гистограмму, показывающую динамику рентабельности по годам. Для этого сначала выполним SPARQL-запрос, чтобы получить данные о рентабельности за *все* годы, а затем используем `matplotlib` для построения графика.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# SPARQL-запрос для получения рентабельности за все годы
q_all_profitability = """
PREFIX schema: <http://example.org/schema#>

SELECT ?year ((?profit / ?income) * 100 AS ?profitability)
WHERE {
    ?report schema:year ?year .
    ?report schema:profit ?profit .
    ?report schema:income ?income .
}
ORDER BY ?year
"""

# Выполняем запрос и собираем результаты
results = g.query(q_all_profitability)

# Преобразуем результаты в DataFrame для удобства
data_for_plot = []
for row in results:
    data_for_plot.append({
        'Год': int(row.year),
        'Рентабельность': float(row.profitability)
    })

df_plot = pd.DataFrame(data_for_plot)

# Строим график
plt.figure(figsize=(10, 6))
plt.bar(df_plot['Год'], df_plot['Рентабельность'], color='skyblue')

# Добавляем подписи и заголовок
plt.xlabel("Год", fontsize=12)
plt.ylabel("Рентабельность (%)", fontsize=12)
plt.title("Динамика рентабельности по годам", fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(df_plot['Год']) # Убедимся, что все годы отображены на оси X
plt.ylim(0, 50) # Устанавливаем предел для оси Y для наглядности

# Показываем график
plt.show()

## Выводы по Части II

Вторая часть практической работы продемонстрировала фундаментально иной подход к представлению и анализу данных по сравнению с традиционными табличными методами, используемыми в Excel. Преобразование экономических показателей в граф знаний и использование языка SPARQL позволило выявить как сильные стороны семантического подхода, так и его особенности.

**1. Гибкость и расширяемость модели данных.**
В отличие от строгой структуры Excel-таблицы, графовая модель оказалась чрезвычайно гибкой. Мы смогли определить сущности («Годовой отчет») и их свойства (`доход`, `прибыль`), не привязываясь к колонкам. Если бы потребовалось добавить новые, неструктурированные данные (например, «ключевое событие года» или «ответственный менеджер»), это можно было бы сделать, просто добавив новые триплеты, не изменяя существующую схему. В Excel это потребовало бы введения новых столбцов, которые для многих строк остались бы пустыми.

**2. Мощь и выразительность аналитических запросов.**
Язык SPARQL позволил выполнять запросы, которые в Excel потребовали бы сложных многоступенчатых фильтров, сортировок и вспомогательных вычислений. Запрос на расчет рентабельности и поиск топ-3 годов был выполнен одной компактной операцией. SPARQL позволяет формулировать вопросы к данным на уровне их смысла и связей («найди мне годы, где X больше Y и связано с Z»), а не на уровне операций с ячейками.

**3. Явное представление знаний.**
Ключевое преимущество графа в том, что отношения между данными (`schema:income`, `rdf:type`) являются полноправными элементами модели. Это делает смысл данных понятным для машинной обработки и позволяет строить системы, способные делать логические выводы. Таблица Excel, напротив, хранит данные, но их семантика и взаимосвязи остаются вне самой таблицы, в голове аналитика.

**4. Наглядность сложных связей.**
Интерактивная визуализация графа показала, что все данные представляют собой единую сеть. Хотя наш пример был простым, в реальных задачах, где десятки сущностей (компании, проекты, сотрудники, рынки) связаны друг с другом, именно графовая визуализация помогает находить неочевидные зависимости и "узкие места".

**Заключение:**
Подход, основанный на графах знаний, является мощным инструментом для аспиранта-исследователя в области менеджмента. Он требует более высокого порога входа по сравнению с Excel, но предоставляет несравнимо большие возможности для глубокого анализа сложных, взаимосвязанных экономических и управленческих систем. Этот метод идеально подходит для интеграции разнородных данных, выявления скрытых паттернов и построения интеллектуальных аналитических систем, что является передовым краем цифровых технологий в экономических исследованиях.

## Выводы и сравнение подходов

В заключительной части отчета проанализируйте проделанную работу.

1.  **Сравните табличный (Excel) и графовый (RDF/SPARQL) подходы.**
    *   **Excel:** в чем его сильные стороны (наглядность, простота базовых операций)? В чем ограничения (сложность запросов, негибкость структуры)?
    *   **RDF/SPARQL:** какие преимущества дает графовая модель (гибкость схемы, возможность описывать сложные связи, мощный язык запросов)? Какие у нее недостатки (более высокий порог входа, избыточность для простых задач)?
2.  **Подумайте, как можно было бы расширить онтологию.** Какие еще сущности и связи можно было бы добавить для более глубокого экономического анализа (например, связать инвестиции с конкретными проектами, добавить данные о сотрудниках, рыночных индексах и т.д.)?